In [1]:
!pip install azureml
!pip install azureml.core
!pip install azureml.widgets
!pip install azureml.pipeline

  Using cached azureml_core-1.43.0-py3-none-any.whl (2.7 MB)
ERROR: azureml-train-automl-runtime 1.42.0 has requirement azureml-automl-core~=1.42.0, but you'll have azureml-automl-core 1.43.0 which is incompatible.
ERROR: azureml-train-automl-runtime 1.42.0 has requirement azureml-core~=1.42.0, but you'll have azureml-core 1.43.0 which is incompatible.
ERROR: azureml-train-automl-runtime 1.42.0 has requirement azureml-dataset-runtime[fuse,pandas]~=1.42.0, but you'll have azureml-dataset-runtime 1.43.0 which is incompatible.
ERROR: azureml-train-automl-runtime 1.42.0 has requirement azureml-telemetry~=1.42.0, but you'll have azureml-telemetry 1.43.0 which is incompatible.
ERROR: azureml-train-automl-runtime 1.42.0 has requirement azureml-train-automl-client~=1.42.0, but you'll have azureml-train-automl-client 1.43.0 which is incompatible.
ERROR: azureml-tensorboard 1.42.0 has requirement azureml-core~=1.42.0, but you'll have azureml-core 1.43.0 which is incompatible.
ERROR: azureml-sdk 

In [2]:
import os
from azureml.core import Workspace, Experiment, Environment, ScriptRunConfig, Dataset, Run
from azureml.widgets import RunDetails
from azureml.pipeline.core import Pipeline, PipelineData
from azureml.pipeline.steps import PythonScriptStep

In [3]:
ws = Workspace.from_config()

In [4]:
keyvault = ws.get_default_keyvault()

In [5]:
default_datastore = ws.get_default_datastore()

In [6]:
dataset_cord19_name = 'cord19'

# Get a dataset by name
dataset_cord19 = Dataset.get_by_name(workspace=ws, name=dataset_cord19_name)

### Create CPU Compute

In [7]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# CPU cluster name and sku
aml_cpu_compute_cluster_name = 'cpuclust-d12v2'
vm_size = 'Standard_D12_v2'

# Check cluster does not exist already
try:
    aml_cpu_compute = ComputeTarget(workspace=ws, name=aml_cpu_compute_cluster_name)
    print('Found existing cluster {}'.format(aml_cpu_compute_cluster_name))
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size=vm_size,
                                                           max_nodes=8)
    aml_cpu_compute = ComputeTarget.create(ws, aml_cpu_compute_cluster_name, compute_config)

aml_cpu_compute.wait_for_completion(show_output=True)

Found existing cluster cpuclust-d12v2
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


### Define Runtime Configuration

In [8]:
from azureml.core.runconfig import RunConfiguration
aml_run_config = RunConfiguration()

from azureml.core.environment import Environment
# Create the environment
env = Environment(name='rp')
# Enable Docker and reference an image
env.docker.enabled = True
env.python.user_managed_dependencies=True

env.docker.base_image_registry.address = "xxxxcr.azurecr.io"
env.docker.base_image_registry.username = keyvault.get_secret(name='xxxxcr-admin-user')
env.docker.base_image_registry.password = keyvault.get_secret(name='xxxxcr-admin-pwd')
env.docker.base_image = xxxxcr.azurecr.io/azureml-env-base-sd:latest"
                          
aml_run_config.environment = env
env.register(workspace=ws)

'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "csssdemocr.azurecr.io/azureml-env-base-sd:latest",
        "baseImageRegistry": {
            "address": "csssdemocr.azurecr.io",
            "password": "AzureMlSecret=Env_796e80c7e5bf8a48cb603f229eeec578ec72443dbfe37710cc80de67228c6713_10#EnvironmentDefinition#ContainerRegistry#Password",
            "registryIdentity": null,
            "username": "csssdemocr"
        },
        "enabled": true,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "rp",
    "python": {
        "baseCondaEn

## Specify script source folder(s)

In [9]:
dataprep_script_folder = ''
print('Source directory for data preparation is {}.'.format(os.path.realpath(dataprep_script_folder)))

Source directory for data preparation is /mnt/batch/tasks/shared/LS_root/mounts/clusters/cpu-std-ds11v2/code/Users/jp/search-demo/etl/cord19.


### Step - Extract

In [10]:
dataset_size_percent = 1
step_dataprep_extract_out = PipelineData('extract_out', datastore=default_datastore)

step_dataprep_extract = PythonScriptStep(
    name='Extract',
    script_name='step_dataprep_extract.py', 
    arguments=['--input', dataset_cord19.as_named_input('raw_data').as_mount(),
               '--dataset_size_percent', dataset_size_percent,
               '--pmc_only', 'n',
               '--output', step_dataprep_extract_out],
    outputs=[step_dataprep_extract_out],
    compute_target=aml_cpu_compute,
    runconfig=aml_run_config,
    source_directory=dataprep_script_folder,
    allow_reuse=True
)

print('Step Extract Created')

Step Extract Created


### Step - Language Detection

In [11]:
step_dataprep_lang_detect_out = PipelineData('lang_detect_out', datastore=default_datastore)

step_dataprep_lang_detect = PythonScriptStep(
    name='Language Detection',
    script_name='step_dataprep_lang_detect.py', 
    arguments=['--input', step_dataprep_extract_out,
               '--max_doc_length', 1000,
               '--output', step_dataprep_lang_detect_out],
    inputs=[step_dataprep_extract_out],
    outputs=[step_dataprep_lang_detect_out],
    compute_target=aml_cpu_compute,
    runconfig=aml_run_config,
    source_directory=dataprep_script_folder,
    allow_reuse=True
)

print('Step Language Detection Created')

Step Language Detection Created


### Step - Create Hash Index

In [12]:
step_dataprep_hash_index_out = PipelineData('hash_index_out', datastore=default_datastore)

step_dataprep_hash_index = PythonScriptStep(
    name='Hash Index',
    script_name='step_dataprep_hash_index.py', 
    arguments=['--input', step_dataprep_lang_detect_out,
               '--output', step_dataprep_hash_index_out],
    inputs=[step_dataprep_lang_detect_out],
    outputs=[step_dataprep_hash_index_out],
    compute_target=aml_cpu_compute,
    runconfig=aml_run_config,
    source_directory=dataprep_script_folder,
    allow_reuse=True
)

print('Step Hash Index Created')

Step Hash Index Created


### Step - Stopwords & Lemmatize

In [13]:
step_dataprep_stopwords_lemma_out = PipelineData('stopwords_lemma_out', datastore=default_datastore)

step_dataprep_stopwords_lemma = PythonScriptStep(
    name='Stopwords & Lemmatization',
    script_name='step_dataprep_stopwords_lemma.py', 
    arguments=['--input', step_dataprep_hash_index_out,
               '--output', step_dataprep_stopwords_lemma_out],
    inputs=[step_dataprep_hash_index_out],
    outputs=[step_dataprep_stopwords_lemma_out],
    compute_target=aml_cpu_compute,
    runconfig=aml_run_config,
    source_directory=dataprep_script_folder,
    allow_reuse=True
)

print('Step Stopwords Lemma Created')

Step Stopwords Lemma Created


### Step - Trials

In [14]:
step_dataprep_trials_out = PipelineData('trials_out', datastore=default_datastore)

step_dataprep_trials = PythonScriptStep(
    name='Trials',
    script_name='step_dataprep_trials.py', 
    arguments=['--input', step_dataprep_hash_index_out,
               '--output', step_dataprep_trials_out],
    inputs=[step_dataprep_hash_index_out],
    outputs=[step_dataprep_trials_out],
    compute_target=aml_cpu_compute,
    runconfig=aml_run_config,
    source_directory=dataprep_script_folder,
    allow_reuse=True
)

print('Step Trials created')

Step Trials created


### Step - Feature Engineering

In [15]:
step_dataprep_feature_engineering_out = PipelineData('feature_engineering_out', datastore=default_datastore)

step_dataprep_feature_engineering = PythonScriptStep(
    name='Feature Engineering',
    script_name='step_dataprep_feature_engineering.py', 
    arguments=['--input', step_dataprep_hash_index_out,
               '--output', step_dataprep_feature_engineering_out],
    inputs=[step_dataprep_hash_index_out],
    outputs=[step_dataprep_feature_engineering_out],
    compute_target=aml_cpu_compute,
    runconfig=aml_run_config,
    source_directory=dataprep_script_folder,
    allow_reuse=True
)

print('Step Feature Engineering created')

Step Feature Engineering created


### Step - Author Article Metrics

In [16]:
step_dataprep_author_article_metrics_out = PipelineData('author_article_metrics_out', datastore=default_datastore)
step_dataprep_author_info_out = PipelineData('author_info_out', datastore=default_datastore)
step_dataprep_authors_by_paper_out = PipelineData('authors_by_paper_out', datastore=default_datastore)

step_dataprep_author_article_metrics = PythonScriptStep(
    name='Author & Article Metrics',
    script_name='step_dataprep_author_article_metrics.py', 
    arguments=['--input', step_dataprep_feature_engineering_out,
               '--output_author_article_metrics', step_dataprep_author_article_metrics_out,
               '--output_author_info', step_dataprep_author_info_out,
               '--output_authors_by_paper', step_dataprep_authors_by_paper_out],
    inputs=[step_dataprep_feature_engineering_out],
    outputs=[step_dataprep_author_article_metrics_out, 
             step_dataprep_author_info_out,
             step_dataprep_authors_by_paper_out],
    compute_target=aml_cpu_compute,
    runconfig=aml_run_config,
    source_directory=dataprep_script_folder,
    allow_reuse=True
)

print('Step Author Article Metrics Created')

Step Author Article Metrics Created


### Step - Topic Modelling

In [17]:
step_dataprep_topic_modelling_topics_out = PipelineData('topic_modelling_topics_out', datastore=default_datastore)
step_dataprep_topic_modelling_document_topic_out = PipelineData('topic_modelling_document_topic_out', datastore=default_datastore)

step_dataprep_topic_modelling = PythonScriptStep(
    name='Topic Modelling',
    script_name='step_dataprep_topic_modelling.py', 
    arguments=['--input', step_dataprep_stopwords_lemma_out,
               '--n_topics', 5,
               '--n_top_words', 20,
               '--no_optimise',
               '--output_topics', step_dataprep_topic_modelling_topics_out,
               '--output_document_topic', step_dataprep_topic_modelling_document_topic_out],
    inputs=[step_dataprep_stopwords_lemma_out],
    outputs=[step_dataprep_topic_modelling_topics_out,
             step_dataprep_topic_modelling_document_topic_out],
    compute_target=aml_cpu_compute,
    runconfig=aml_run_config,
    source_directory=dataprep_script_folder,
    allow_reuse=True
)

print('Step Topic Modelling Created')

Step Topic Modelling Created


### Step - Database Update

In [18]:
step_database_update = PipelineData('database_update', datastore=default_datastore)

step_database_update = PythonScriptStep(
    name='Database Update',
    script_name='step_database_update.py',
    arguments=['--input_topic_modelling_topics', step_dataprep_topic_modelling_topics_out,
               '--input_topic_modelling_document_topic', step_dataprep_topic_modelling_document_topic_out,
               '--input_feature_engineering', step_dataprep_feature_engineering_out,
               '--input_author_info', step_dataprep_author_info_out,
               '--input_author_article_metrics', step_dataprep_author_article_metrics_out,
               '--input_authors_by_paper', step_dataprep_authors_by_paper_out,
               '--input_trials', step_dataprep_trials_out],
    inputs=[step_dataprep_topic_modelling_topics_out,
            step_dataprep_topic_modelling_document_topic_out,
            step_dataprep_feature_engineering_out,
            step_dataprep_author_info_out,
            step_dataprep_author_article_metrics_out,
            step_dataprep_authors_by_paper_out,
            step_dataprep_trials_out],
    compute_target=aml_cpu_compute,
    runconfig=aml_run_config,
    source_directory=dataprep_script_folder,
    allow_reuse=True
)

print('Step Database Update Created')

Step Database Update Created


### Create Step List

In [19]:
steps = [step_dataprep_extract, 
         step_dataprep_lang_detect, 
         step_dataprep_hash_index,
         step_dataprep_stopwords_lemma, 
         step_dataprep_author_article_metrics, 
         step_dataprep_feature_engineering, 
         step_dataprep_topic_modelling,
         step_dataprep_trials,
         step_database_update]
print("Step lists created")

Step lists created


In [20]:
aml_pipeline = Pipeline(workspace=ws, steps=steps)
print ('Pipeline is built')

Pipeline is built


In [21]:
aml_pipeline.validate()
print('Pipeline validation complete')

Step Extract is ready to be created [b6bd9956]Step Language Detection is ready to be created [239e0426]

Step Hash Index is ready to be created [88f35794]
Step Stopwords & Lemmatization is ready to be created [d1a4b73e]
Step Author & Article Metrics is ready to be created [ed95f2b2]Step Feature Engineering is ready to be created [197ff05f]
Step Topic Modelling is ready to be created [20a7dfee]

Step Trials is ready to be created [4acb7e84]
Step Database Update is ready to be created [7e8e1df7]
Pipeline validation complete


In [22]:
# Note regenerate output, decides whether to execute the module to regenerate output at running time.
# It's by default unselected, which means if the module has been executed with the same parameters previously, 
# the system will reuse the output from last run to reduce run time.

aml_pipeline_run = Experiment(ws, 'pipeline-cord19-dataprep').submit(aml_pipeline, regenerate_outputs=False)
print('Experiment pipeline cord19 dataprep is submitted for execution')

Created step Extract [b6bd9956][1008c45b-5db8-436d-b3e4-be3b2cd97dbe], (This step will run and generate new outputs)Created step Language Detection [239e0426][d634f520-a90a-42c1-99e2-5f77fdd6a747], (This step will run and generate new outputs)

Created step Hash Index [88f35794][d78da817-0993-47ae-88c5-d96bdc0ce462], (This step will run and generate new outputs)
Created step Stopwords & Lemmatization [d1a4b73e][0efc6516-7555-4887-a9c5-df68cde4be71], (This step will run and generate new outputs)
Created step Author & Article Metrics [ed95f2b2][938a155b-723e-4e59-9928-8bd68b7df180], (This step will run and generate new outputs)
Created step Feature Engineering [197ff05f][a8d3a3b9-7ba0-4051-8055-19835b38ac8a], (This step will run and generate new outputs)
Created step Topic Modelling [20a7dfee][6fa16ba0-36d4-4d11-b84c-e082af205f9a], (This step will run and generate new outputs)
Created step Trials [4acb7e84][2ce3a0af-6549-47ed-8ccd-c770c8f3fdc8], (This step will run and generate new outpu

In [23]:
RunDetails(aml_pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

In [24]:
step_runs = aml_pipeline_run.get_children()
for step_run in step_runs:
    status = step_run.get_status()
    print('Script:', step_run.name, 'status:', status)
    
    # Change this if you want to see details even if the Step has succeeded.
    if status == "Failed":
        joblog = step_run.get_job_log()
        print('job log:', joblog)